# P1

In [5]:
# k
# -*- coding: utf-8 -*-
"""
bidmc_combined_final_NO_NORM_CONDITIONS.ipynb

Phiên bản kết hợp, nhưng KHÔNG chuẩn hóa điều kiện HR/RR.
"""

import numpy as np
import pandas as pd
import scipy.io as sio
import tensorflow as tf
import matplotlib.pyplot as plt
import pywt
# import joblib # Không cần joblib nữa vì không lưu scaler

# from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler # Chỉ cần MinMaxScaler cho PPG
from sklearn.metrics import mean_squared_error

from scipy.signal import butter, filtfilt, welch, iirnotch
from scipy.fft import fft, fftfreq

from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard

import datetime
import time
import sys
import os

# Dữ liệu gốc lưu ở
data_path = r'data/bidmc_data.mat'

# Đường dẫn đến file dữ liệu (nên tạo thư mục mới cho phiên bản này)
output_base_path = r'bidmc/combined_model_output_no_norm' # Thư mục mới
processed_data_path = os.path.join(output_base_path, 'processed')
figures_path = os.path.join(output_base_path, 'figures')
results_path = os.path.join(output_base_path, 'results')
model_path = os.path.join(output_base_path, 'models')

# Tạo thư mục nếu chưa tồn tại
os.makedirs(processed_data_path, exist_ok=True)
os.makedirs(figures_path, exist_ok=True)
os.makedirs(results_path, exist_ok=True)
os.makedirs(model_path, exist_ok=True)

print("Thiết lập đường dẫn và thư mục hoàn tất.")

# --- Phần Explore Data (Giữ nguyên) ---
print("Đang tải dữ liệu từ file .mat...")
try:
    mat_data = sio.loadmat(data_path)
    data = mat_data['data'][0]
    print(f"Số lượng bản ghi: {len(data)}")
except Exception as e:
    print(f"Lỗi khi tải hoặc khám phá dữ liệu: {e}")
# --- Hết Phần Explore ---


# --- Phần Preprocess Data ---
print("\nBắt đầu Tiền xử lý dữ liệu (KHÔNG chuẩn hóa HR/RR)...")

# Hàm chuẩn hóa tín hiệu PPG (MinMax về [0, 1])
def normalize_signal_minmax(signal):
    scaler = MinMaxScaler(feature_range=(0, 1))
    signal_reshaped = signal.reshape(-1, 1)
    normalized = scaler.fit_transform(signal_reshaped).flatten()
    return normalized

# Các hàm lọc (Giữ nguyên)
def notch_filter(data, notch_freq=50.0, fs=125, quality_factor=30):
    nyq = 0.5 * fs
    w0 = notch_freq / nyq
    if w0 >= 1.0:
         print(f"Warning: Notch frequency {notch_freq}Hz is too high for sampling rate {fs}Hz. Skipping notch filter.")
         return data
    b, a = iirnotch(w0, quality_factor)
    return filtfilt(b, a, data)

def butter_bandpass_filter(data, lowcut=0.5, highcut=8.0, fs=125, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    low = max(0.01, low)
    high = min(0.99, high)
    if low >= high:
        print(f"Warning: Invalid frequency range [{lowcut}, {highcut}] for sampling rate {fs}. Skipping bandpass filter.")
        return data
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, data)

# Hàm chia đoạn (Giữ nguyên)
def segment_signal(signal, segment_length, overlap=0.5):
    step = int(segment_length * (1 - overlap))
    segments = []
    if step <= 0: step = 1
    for i in range(0, len(signal) - segment_length + 1, step):
        segments.append(signal[i:i + segment_length])
    if not segments:
        print(f"Warning: Could not create segments for signal of length {len(signal)} with segment_length {segment_length} and overlap {overlap}.")
    return np.array(segments)

# Hàm trích xuất HR/RR trung bình (Giữ nguyên)
def extract_mean_hr_rr(record):
    hr_values_list = []
    rr_values_list = []
    try:
        params = record['ref'][0, 0]['params'][0, 0]
        hr_data = params['hr'][0]
        rr_data = params['rr'][0]
        # Xử lý HR
        hr_values_raw = hr_data['v'] if hasattr(hr_data, 'dtype') and 'v' in hr_data.dtype.names else hr_data
        for item in hr_values_raw:
             val = item[0] if isinstance(item, (list, np.ndarray)) and len(item) > 0 else item
             if np.isscalar(val) and not np.isnan(val): hr_values_list.append(float(val))
        # Xử lý RR
        rr_values_raw = rr_data['v'] if hasattr(rr_data, 'dtype') and 'v' in rr_data.dtype.names else rr_data
        for item in rr_values_raw:
            val = item[0] if isinstance(item, (list, np.ndarray)) and len(item) > 0 else item
            if np.isscalar(val) and not np.isnan(val): rr_values_list.append(float(val))
        if not hr_values_list or not rr_values_list: return None, None
        return np.mean(hr_values_list), np.mean(rr_values_list)
    except Exception as e: return None, None

# Tham số tiền xử lý (Giữ nguyên)
fs = 125
segment_length = 8 * fs # 1000 samples
overlap = 0.5
lowcut = 0.5
highcut = 8.0
notch_freq = 50.0

# Danh sách lưu trữ
all_ppg_segments = []
all_hr_conditions_raw = [] # Lưu giá trị gốc
all_rr_conditions_raw = [] # Lưu giá trị gốc

valid_records_count = 0
# Vòng lặp xử lý từng bản ghi
for i in range(len(data)):
    try:
        record = data[i]
        ppg_signal_raw = record['ppg'][0, 0]['v'].flatten().astype(np.float64)
        hr_mean, rr_mean = extract_mean_hr_rr(record)
        if hr_mean is None or rr_mean is None:
            # print(f"Skipping record {i}: Insufficient HR/RR data.") # Bỏ qua log này để đỡ rối
            continue

        ppg_notched = notch_filter(ppg_signal_raw, notch_freq=notch_freq, fs=fs)
        ppg_filtered = butter_bandpass_filter(ppg_notched, lowcut=lowcut, highcut=highcut, fs=fs)
        ppg_normalized = normalize_signal_minmax(ppg_filtered)
        segments = segment_signal(ppg_normalized, segment_length, overlap)

        if segments.size > 0:
            num_segments = segments.shape[0]
            all_ppg_segments.extend(segments)
            all_hr_conditions_raw.extend([hr_mean] * num_segments) # Lưu giá trị gốc
            all_rr_conditions_raw.extend([rr_mean] * num_segments) # Lưu giá trị gốc
            valid_records_count += 1
        # else: print(f"Skipping record {i}: No segments generated after processing.") # Bỏ qua log

    except Exception as e:
        print(f"Error processing record {i}: {e}")

print(f"\nĐã xử lý thành công {valid_records_count}/{len(data)} bản ghi.")

if not all_ppg_segments:
    raise ValueError("Không có dữ liệu nào được xử lý thành công.")

# Chuyển thành Numpy arrays
X_data = np.array(all_ppg_segments).astype(np.float32)
# Giữ lại giá trị HR/RR gốc
hr_data_raw = np.array(all_hr_conditions_raw).astype(np.float32).reshape(-1, 1)
rr_data_raw = np.array(all_rr_conditions_raw).astype(np.float32).reshape(-1, 1)

print(f"Tổng số đoạn tín hiệu PPG: {X_data.shape[0]}")

# Chia train/test (90/10) - Sử dụng trực tiếp giá trị gốc
X_train, X_test, hr_train, hr_test, rr_train, rr_test = train_test_split(
    X_data, hr_data_raw, rr_data_raw, test_size=0.1, random_state=42
)

print(f"Kích thước tập huấn luyện: {X_train.shape[0]}")
print(f"Kích thước tập kiểm thử: {X_test.shape[0]}")

# ---> KHÔNG CẦN CHUẨN HÓA ĐIỀU KIỆN NỮA <---
# print("\nBỏ qua bước chuẩn hóa điều kiện HR và RR.")

# Tạo mảng điều kiện cuối cùng từ giá trị gốc
condition_train = np.column_stack((hr_train.flatten(), rr_train.flatten())).astype(np.float32)
condition_test = np.column_stack((hr_test.flatten(), rr_test.flatten())).astype(np.float32)

print("Đã kết hợp điều kiện HR/RR (giá trị gốc).")
print(f"Shape condition_train: {condition_train.shape}")
print(f"Shape condition_test: {condition_test.shape}")

# Lưu dữ liệu đã xử lý
np.save(os.path.join(processed_data_path, 'X_train.npy'), X_train)
np.save(os.path.join(processed_data_path, 'X_test.npy'), X_test)
np.save(os.path.join(processed_data_path, 'condition_train_raw.npy'), condition_train) # Lưu điều kiện gốc
np.save(os.path.join(processed_data_path, 'condition_test_raw.npy'), condition_test)   # Lưu điều kiện gốc
# Lưu thêm HR/RR riêng lẻ nếu cần
np.save(os.path.join(processed_data_path, 'hr_train_raw.npy'), hr_train)
np.save(os.path.join(processed_data_path, 'hr_test_raw.npy'), hr_test)
np.save(os.path.join(processed_data_path, 'rr_train_raw.npy'), rr_train)
np.save(os.path.join(processed_data_path, 'rr_test_raw.npy'), rr_test)
print("Đã lưu các file dữ liệu đã xử lý vào:", processed_data_path)

print("\nTiền xử lý dữ liệu hoàn tất (HR/RR không được chuẩn hóa).")
# --- Hết Phần Preprocess ---


# --- Phần CVAE Model Definition (Giữ nguyên định nghĩa kiến trúc) ---
print("\nĐịnh nghĩa mô hình CVAE...")

# Tham số mô hình
input_dim = 1000 # Giả định segment_length = 1000
condition_dim = 2
latent_dim = 32
batch_size = 64
epochs = 500
beta_max = 1.5
warmup_epochs = 50
learning_rate = 0.0005

# Lớp Sampling (Giữ nguyên)
class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# Encoder (Giữ nguyên)
def build_encoder(input_dim, condition_dim, latent_dim, hidden_units_dense=[256, 128]):
    encoder_inputs = layers.Input(shape=(input_dim,), name='encoder_input')
    x = layers.Reshape((input_dim, 1))(encoder_inputs)
    x = layers.Conv1D(filters=64, kernel_size=5, strides=1, activation='swish', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Conv1D(filters=128, kernel_size=5, strides=1, activation='swish', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Conv1D(filters=256, kernel_size=5, strides=1, activation='swish', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    shape_before_flattening = tf.keras.backend.int_shape(x)[1:]
    x = layers.Flatten()(x)
    condition_inputs = layers.Input(shape=(condition_dim,), name='condition_input')
    cond_dense = layers.Dense(16, activation='relu')(condition_inputs)
    x = layers.Concatenate()([x, cond_dense])
    for units in hidden_units_dense:
        x = layers.Dense(units, activation='swish')(x)
        x = layers.Dropout(0.3)(x)
    z_mean = layers.Dense(latent_dim, name='z_mean')(x)
    z_log_var = layers.Dense(latent_dim, name='z_log_var')(x)
    z = Sampling()([z_mean, z_log_var])
    encoder = Model([encoder_inputs, condition_inputs], [z_mean, z_log_var, z], name='encoder')
    return encoder, shape_before_flattening

# Decoder (Giữ nguyên, vẫn dùng sigmoid output để khớp PPG chuẩn hóa [0,1])
def build_decoder(latent_dim, condition_dim, input_dim, shape_before_flattening, hidden_units_dense=[128, 256]):
    latent_inputs = layers.Input(shape=(latent_dim,), name='latent_input')
    condition_inputs = layers.Input(shape=(condition_dim,), name='condition_input')
    cond_dense = layers.Dense(16, activation='relu')(condition_inputs)
    x = layers.Concatenate()([latent_inputs, cond_dense])
    for units in reversed(hidden_units_dense):
         x = layers.Dense(units, activation='swish')(x)
    target_shape_units = np.prod(shape_before_flattening)
    x = layers.Dense(target_shape_units, activation='swish')(x)
    x = layers.Reshape(shape_before_flattening)(x)
    x = layers.Conv1DTranspose(128, kernel_size=5, strides=2, padding='same', activation='swish')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1DTranspose(64, kernel_size=5, strides=2, padding='same', activation='swish')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1DTranspose(1, kernel_size=5, strides=2, padding='same', activation='sigmoid')(x) # Giữ sigmoid
    decoder_outputs = layers.Reshape((input_dim,))(x)
    decoder = Model([latent_inputs, condition_inputs], decoder_outputs, name='decoder')
    return decoder

# Lớp CVAE (Giữ nguyên logic KL Annealing)
class CVAE_Combined(Model):
    # ... (Giữ nguyên toàn bộ định nghĩa lớp CVAE_Combined như phiên bản trước) ...
    def __init__(self, encoder, decoder, latent_dim, beta_max, warmup_epochs, **kwargs):
        super(CVAE_Combined, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.latent_dim = latent_dim
        self.beta_max = tf.constant(beta_max, dtype=tf.float32)
        self.warmup_epochs = tf.constant(warmup_epochs, dtype=tf.float32)
        self.epoch_counter = tf.Variable(0.0, trainable=False, name="epoch_counter", dtype=tf.float32)
        self.total_loss_tracker = tf.keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = tf.keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = tf.keras.metrics.Mean(name="kl_loss")
        self.beta_tracker = tf.keras.metrics.Mean(name="beta")

    @property
    def metrics(self):
        return [ self.total_loss_tracker, self.reconstruction_loss_tracker, self.kl_loss_tracker, self.beta_tracker ]

    def get_current_beta(self):
         safe_warmup_epochs = tf.maximum(self.warmup_epochs, 1.0)
         current_progress = (self.epoch_counter + 1.0) / safe_warmup_epochs
         beta = tf.minimum(self.beta_max, self.beta_max * current_progress)
         return beta

    def train_step(self, data):
        x, condition = data
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder([x, condition], training=True)
            reconstruction = self.decoder([z, condition], training=True)
            reconstruction_loss = tf.reduce_mean(tf.reduce_sum(tf.keras.losses.mse(x, reconstruction), axis=1))
            kl_loss = -0.5 * tf.reduce_mean(tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=1))
            current_beta = self.get_current_beta()
            total_loss = reconstruction_loss + current_beta * kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        self.beta_tracker.update_state(current_beta)
        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        x, condition = data
        z_mean, z_log_var, z = self.encoder([x, condition], training=False)
        reconstruction = self.decoder([z, condition], training=False)
        reconstruction_loss = tf.reduce_mean(tf.reduce_sum(tf.keras.losses.mse(x, reconstruction), axis=1))
        kl_loss = -0.5 * tf.reduce_mean(tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=1))
        current_beta = self.get_current_beta()
        total_loss = reconstruction_loss + current_beta * kl_loss
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        self.beta_tracker.update_state(current_beta)
        return {m.name: m.result() for m in self.metrics}

    def generate(self, condition, z=None, noise_scale=1.0):
         if z is None: z = tf.random.normal(shape=(tf.shape(condition)[0], self.latent_dim)) * noise_scale
         if tf.rank(condition) == 1: condition = tf.expand_dims(condition, axis=0)
         return self.decoder([z, condition], training=False)

# Callback cập nhật epoch (Giữ nguyên)
class EpochCounterCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        if hasattr(self.model, 'epoch_counter'):
            self.model.epoch_counter.assign(tf.cast(epoch, tf.float32))

# Xây dựng mô hình
print("\nXây dựng Encoder và Decoder...")
encoder, shape_enc = build_encoder(input_dim, condition_dim, latent_dim)
decoder = build_decoder(latent_dim, condition_dim, input_dim, shape_enc)
# encoder.summary(line_length=100) # Bỏ summary để đỡ dài
# decoder.summary(line_length=100)

print("\nXây dựng và biên dịch mô hình CVAE...")
cvae_model_no_norm = CVAE_Combined(encoder, decoder, latent_dim, beta_max, warmup_epochs)
cvae_model_no_norm.compile(optimizer=Adam(learning_rate=learning_rate))
print("Biên dịch hoàn tất.")

# --- Hết Phần Model Definition ---


# --- Phần Training ---
print("\nBắt đầu Huấn luyện mô hình CVAE (HR/RR không chuẩn hóa)...")

# Load dữ liệu đã xử lý (X_train, condition_train_raw)
X_train = np.load(os.path.join(processed_data_path, 'X_train.npy'))
# LƯU Ý: Load condition_train_raw.npy chứa giá trị gốc
condition_train = np.load(os.path.join(processed_data_path, 'condition_train_raw.npy'))
# Load dữ liệu test tương ứng
X_test = np.load(os.path.join(processed_data_path, 'X_test.npy'))
condition_test = np.load(os.path.join(processed_data_path, 'condition_test_raw.npy'))


# Thiết lập Callbacks (Giữ nguyên như phiên bản kết hợp)
log_dir = os.path.join(model_path, "logs_no_norm", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
checkpoint_path = os.path.join(model_path, "cvae_no_norm_cond_best.weights.h5") # Tên file mới
checkpoint_callback = ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, monitor='val_total_loss', mode='min', save_best_only=True, verbose=1)
early_stopping_callback = EarlyStopping(monitor='val_total_loss', patience=50, restore_best_weights=True, verbose=1)
reduce_lr_callback = ReduceLROnPlateau(monitor='val_total_loss', factor=0.2, patience=20, min_lr=1e-7, verbose=1)
combined_callbacks = [EpochCounterCallback(), tensorboard_callback, checkpoint_callback, early_stopping_callback, reduce_lr_callback]

# Tạo dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, condition_train)).shuffle(buffer_size=X_train.shape[0]).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, condition_test)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Huấn luyện
start_time = time.time()
history = cvae_model_no_norm.fit(
    train_dataset,
    epochs=epochs,
    validation_data=test_dataset,
    callbacks=combined_callbacks
)
training_time = time.time() - start_time
print(f"\nHuấn luyện hoàn tất trong {training_time:.2f} giây.")
print(f"Trọng số tốt nhất đã được lưu tại (hoặc khôi phục): {checkpoint_path}")

# Vẽ biểu đồ huấn luyện (Giữ nguyên)
# ... (Code vẽ biểu đồ giống phiên bản trước) ...
print('\nVẽ biểu đồ quá trình huấn luyện...')
plt.figure(figsize=(20, 5))
plt.subplot(1, 5, 1); plt.plot(history.history['total_loss'], label='Train'); plt.plot(history.history['val_total_loss'], label='Val'); plt.title('Total Loss'); plt.legend(); plt.grid(True, alpha=0.3)
plt.subplot(1, 5, 2); plt.plot(history.history['reconstruction_loss'], label='Train'); plt.plot(history.history['val_reconstruction_loss'], label='Val'); plt.title('Recon Loss'); plt.legend(); plt.grid(True, alpha=0.3)
plt.subplot(1, 5, 3); plt.plot(history.history['kl_loss'], label='Train'); plt.plot(history.history['val_kl_loss'], label='Val'); plt.title('KL Loss'); plt.legend(); plt.grid(True, alpha=0.3)
plt.subplot(1, 5, 4); beta_key = 'val_beta' if 'val_beta' in history.history else 'beta'; plt.plot(history.history.get(beta_key, []), label='Beta'); plt.title('Beta'); plt.legend(); plt.grid(True, alpha=0.3)
plt.subplot(1, 5, 5); plt.plot(history.history.get('lr', []), label='LR'); plt.title('Learning Rate'); plt.legend(); plt.grid(True, alpha=0.3)
plt.tight_layout(); plt.savefig(os.path.join(figures_path, 'training_history_no_norm.png')); plt.show(); plt.close()
print("Đã lưu biểu đồ huấn luyện.")
# --- Hết Phần Training ---


# --- Phần Test & Evaluation ---
print("\nBắt đầu Kiểm tra và Đánh giá mô hình (HR/RR không chuẩn hóa)...")

# Load trọng số tốt nhất
if os.path.exists(checkpoint_path):
    print(f"Đang tải trọng số tốt nhất từ: {checkpoint_path}")
    try:
        _ = cvae_model_no_norm([X_test[:1], condition_test[:1]]) # Build nếu cần
        cvae_model_no_norm.load_weights(checkpoint_path)
        print("Đã tải trọng số thành công.")
    except Exception as e: print(f"Lỗi khi tải trọng số: {e}.")
else: print(f"Không tìm thấy file checkpoint: {checkpoint_path}.")

# Tái tạo tín hiệu từ tập Test (dùng condition_test gốc)
num_samples_to_show = 10
X_test_subset = X_test[:num_samples_to_show]
condition_test_subset = condition_test[:num_samples_to_show] # Điều kiện gốc
# Load HR/RR gốc để hiển thị
hr_test_raw_subset = np.load(os.path.join(processed_data_path, 'hr_test_raw.npy'))[:num_samples_to_show]
rr_test_raw_subset = np.load(os.path.join(processed_data_path, 'rr_test_raw.npy'))[:num_samples_to_show]

print("Tái tạo tín hiệu từ tập test...")
z_mean, _, z = cvae_model_no_norm.encoder.predict([X_test_subset, condition_test_subset])
reconstructed_ppg = cvae_model_no_norm.decoder.predict([z, condition_test_subset])

# Vẽ so sánh tín hiệu gốc và tái tạo (Giữ nguyên code vẽ)
plt.figure(figsize=(15, 4 * num_samples_to_show // 2))
for i in range(num_samples_to_show):
    plt.subplot(num_samples_to_show // 2, 2, i + 1)
    plt.plot(X_test_subset[i], label='Original', alpha=0.8)
    plt.plot(reconstructed_ppg[i], label='Reconstructed', alpha=0.8, linestyle='--')
    plt.title(f'Test Sample {i+1} (HR={hr_test_raw_subset[i,0]:.1f}, RR={rr_test_raw_subset[i,0]:.1f})')
    plt.xlabel('Sample'); plt.ylabel('Amplitude'); plt.legend(); plt.grid(True, alpha=0.3)
plt.tight_layout(); plt.savefig(os.path.join(figures_path, 'reconstruction_comparison_no_norm.png')); plt.show(); plt.close()


# Phân tích tần số (Giữ nguyên code phân tích và vẽ)
# ... (Code FFT và vẽ phổ tần số) ...
print("Phân tích phổ tần số...")
def analyze_frequency_spectrum(signal, fs):
    n = len(signal)
    if n == 0: return np.array([]), np.array([])
    yf = fft(signal)
    xf = fftfreq(n, 1/fs)[:n//2]; yf_abs = 2.0/n * np.abs(yf[0:n//2])
    return xf, yf_abs
num_fft_samples = 5
plt.figure(figsize=(15, 5 * num_fft_samples))
for i in range(num_fft_samples):
    xf_orig, yf_orig = analyze_frequency_spectrum(X_test_subset[i], fs)
    xf_recon, yf_recon = analyze_frequency_spectrum(reconstructed_ppg[i], fs)
    plt.subplot(num_fft_samples, 2, 2 * i + 1)
    if xf_orig.size > 0: plt.plot(xf_orig, yf_orig); peaks_orig_idx=np.argsort(yf_orig)[-3:]; plt.plot(xf_orig[peaks_orig_idx], yf_orig[peaks_orig_idx], 'ro', label='Peaks'); [plt.text(xf_orig[idx], yf_orig[idx], f'{xf_orig[idx]:.2f}Hz', fontsize=9) for idx in peaks_orig_idx]
    plt.title(f'Original FFT (Sample {i+1})'); plt.xlabel('Freq (Hz)'); plt.ylabel('Amp'); plt.xlim(0, 10); plt.grid(True, alpha=0.3); plt.legend()
    plt.subplot(num_fft_samples, 2, 2 * i + 2)
    if xf_recon.size > 0: plt.plot(xf_recon, yf_recon, color='orange'); peaks_recon_idx = np.argsort(yf_recon)[-3:]; plt.plot(xf_recon[peaks_recon_idx], yf_recon[peaks_recon_idx], 'ro', label='Peaks'); [plt.text(xf_recon[idx], yf_recon[idx], f'{xf_recon[idx]:.2f}Hz', fontsize=9) for idx in peaks_recon_idx]
    plt.title(f'Reconstructed FFT (Sample {i+1})'); plt.xlabel('Freq (Hz)'); plt.ylabel('Amp'); plt.xlim(0, 10); plt.grid(True, alpha=0.3); plt.legend()
plt.tight_layout(); plt.savefig(os.path.join(figures_path, 'fft_comparison_no_norm.png')); plt.show(); plt.close()

# Tính toán Metrics (Giữ nguyên logic tính toán)
# ... (Code tính MSE, PSNR, Corr và lưu summary) ...
print("Tính toán các chỉ số đánh giá...")
mse_list, psnr_list, corr_list = [], [], []
def calculate_psnr(original, generated, max_pixel=1.0): mse=mean_squared_error(original,generated); return float('inf') if mse==0 else 20*np.log10(max_pixel/np.sqrt(mse))
def calculate_corr(original, generated): return 0.0 if np.std(original)==0 or np.std(generated)==0 else np.corrcoef(original, generated)[0,1]
num_eval_samples = min(1000, X_test.shape[0]); X_eval = X_test[:num_eval_samples]; cond_eval = condition_test[:num_eval_samples]
print(f"Tái tạo {num_eval_samples} mẫu để đánh giá...")
_, _, eval_z = cvae_model_no_norm.encoder.predict([X_eval, cond_eval])
eval_reconstructed = cvae_model_no_norm.decoder.predict([eval_z, cond_eval])
for i in range(num_eval_samples): orig=X_eval[i]; recon=eval_reconstructed[i]; mse_list.append(mean_squared_error(orig, recon)); psnr_list.append(calculate_psnr(orig, recon)); corr_list.append(calculate_corr(orig, recon))
corr_list_valid = [c for c in corr_list if not np.isnan(c)]
results_summary = {"MSE":{"Mean":np.mean(mse_list),"Std":np.std(mse_list),"Min":np.min(mse_list),"Max":np.max(mse_list)}, "PSNR":{"Mean":np.mean(psnr_list),"Std":np.std(psnr_list),"Min":np.min(psnr_list),"Max":np.max(psnr_list)}, "Correlation":{"Mean":np.mean(corr_list_valid),"Std":np.std(corr_list_valid),"Min":np.min(corr_list_valid),"Max":np.max(corr_list_valid)}}
print("\n--- Kết quả Đánh giá Định lượng ---"); [print(f"{m}: Mean={s['Mean']:.4f} +/- {s['Std']:.4f} (Min={s['Min']:.4f}, Max={s['Max']:.4f})") for m, s in results_summary.items()]
results_file_path = os.path.join(results_path, "evaluation_summary_no_norm.txt");
with open(results_file_path, "w") as f: f.write("KẾT QUẢ ĐÁNH GIÁ MÔ HÌNH CVAE (KHÔNG CHUẨN HÓA HR/RR)\n=======================================\n\n"); [f.write(f"{m}:\n  - Mean: {s['Mean']:.4f}\n  - Std Dev: {s['Std']:.4f}\n  - Min: {s['Min']:.4f}\n  - Max: {s['Max']:.4f}\n\n") for m, s in results_summary.items()]
print(f"Đã lưu kết quả đánh giá vào: {results_file_path}")
# --- Hết Phần Test & Evaluation ---


# --- Phần Prediction (Sinh tín hiệu mới - Dùng HR/RR gốc) ---
print("\nBắt đầu Sinh tín hiệu PPG mới (Dùng HR/RR gốc)...")

# ---> KHÔNG CẦN LOAD SCALER NỮA <---
# print("Bỏ qua bước load scaler.")

# Tạo lưới các giá trị HR, RR gốc mong muốn (Giữ nguyên)
hr_raw_values = np.linspace(60, 120, 5)
rr_raw_values = np.linspace(8, 20, 5)

# ---> KHÔNG CẦN CHUẨN HÓA GIÁ TRỊ MỚI <---
# print("Sử dụng trực tiếp giá trị HR/RR gốc.")

# Tạo các cặp điều kiện trực tiếp từ giá trị gốc
conditions_pred_list_raw = []
conditions_raw_list = [] # Vẫn lưu để hiển thị title
for hr_r in hr_raw_values:
    for rr_r in rr_raw_values:
        conditions_pred_list_raw.append([hr_r, rr_r]) # Dùng giá trị gốc
        conditions_raw_list.append([hr_r, rr_r])

conditions_pred_tensor = tf.constant(conditions_pred_list_raw, dtype=tf.float32)
conditions_raw_array = np.array(conditions_raw_list)

print(f"\nSinh {conditions_pred_tensor.shape[0]} tín hiệu PPG với các điều kiện HR/RR gốc...")
# Sinh tín hiệu
generated_ppg_specific = cvae_model_no_norm.generate(conditions_pred_tensor, noise_scale=0.7).numpy()

# Vẽ tín hiệu đã tạo (Giữ nguyên code vẽ)
num_pred_to_show = min(10, generated_ppg_specific.shape[0])
indices_to_show = np.random.choice(generated_ppg_specific.shape[0], num_pred_to_show, replace=False)
plt.figure(figsize=(15, 4 * num_pred_to_show // 2))
plot_count = 1
for idx in indices_to_show:
     if plot_count > num_pred_to_show: break
     plt.subplot(num_pred_to_show // 2, 2, plot_count)
     plt.plot(generated_ppg_specific[idx])
     hr_orig = conditions_raw_array[idx, 0]
     rr_orig = conditions_raw_array[idx, 1]
     plt.title(f'Generated PPG (Target HR={hr_orig:.1f}, RR={rr_orig:.1f})')
     plt.xlabel('Sample'); plt.ylabel('Amplitude'); plt.grid(True, alpha=0.3)
     plot_count += 1
plt.tight_layout(); plt.savefig(os.path.join(figures_path, 'generated_signals_specific_conditions_no_norm.png')); plt.show(); plt.close()
print("Đã lưu biểu đồ các tín hiệu sinh ra.")

# --- Hết Phần Prediction ---

print("\n--- QUÁ TRÌNH HOÀN TẤT (KHÔNG CHUẨN HÓA HR/RR) ---")

Thiết lập đường dẫn và thư mục hoàn tất.
Đang tải dữ liệu từ file .mat...
Số lượng bản ghi: 53

Bắt đầu Tiền xử lý dữ liệu (KHÔNG chuẩn hóa HR/RR)...

Đã xử lý thành công 0/53 bản ghi.


ValueError: Không có dữ liệu nào được xử lý thành công.

# P 2

In [6]:
# code ket hop co chuan hoa BR HR.ipynb
# -*- coding: utf-8 -*-
"""
bidmc_combined_final.ipynb

Kết hợp các cải tiến và sửa lỗi từ các phiên bản trước.
"""

import numpy as np
import pandas as pd
import scipy.io as sio
import tensorflow as tf
import matplotlib.pyplot as plt
import pywt # Thư viện Wavelet
import joblib # Lưu/tải scaler

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler # Thêm StandardScaler
from sklearn.metrics import mean_squared_error

from scipy.signal import butter, filtfilt, welch, iirnotch # Thêm iirnotch
from scipy.fft import fft, fftfreq

from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard # Import callbacks đầy đủ

import datetime
import time
import sys
import os


# Dữ liệu gốc lưu ở
data_path = r'data\bidmc_data.mat'

# Đường dẫn đến file dữ liệu (nên tạo thư mục mới cho phiên bản này)
output_base_path = 'bidmc/combined_model_output'
processed_data_path = os.path.join(output_base_path, 'processed')
figures_path = os.path.join(output_base_path, 'figures')
results_path = os.path.join(output_base_path, 'results')
model_path = os.path.join(output_base_path, 'models')

# Tạo thư mục nếu chưa tồn tại
os.makedirs(processed_data_path, exist_ok=True)
os.makedirs(figures_path, exist_ok=True)
os.makedirs(results_path, exist_ok=True)
os.makedirs(model_path, exist_ok=True)

print("Thiết lập đường dẫn và thư mục hoàn tất.")

# --- Phần Explore Data (Giữ nguyên hoặc tùy chỉnh nếu cần) ---
# (Code khám phá dữ liệu từ các phiên bản trước có thể đặt ở đây)
print("Đang tải dữ liệu từ file .mat...")
try:
    mat_data = sio.loadmat(data_path)
    data = mat_data['data'][0]
    print(f"Số lượng bản ghi: {len(data)}")
    # (Thêm code khám phá chi tiết nếu muốn)
except Exception as e:
    print(f"Lỗi khi tải hoặc khám phá dữ liệu: {e}")
    # Có thể dừng script hoặc xử lý lỗi khác ở đây
# --- Hết Phần Explore ---


# --- Phần Preprocess Data ---
print("\nBắt đầu Tiền xử lý dữ liệu...")

# Hàm chuẩn hóa tín hiệu PPG (MinMax về [0, 1])
def normalize_signal_minmax(signal):
    scaler = MinMaxScaler(feature_range=(0, 1))
    signal_reshaped = signal.reshape(-1, 1)
    normalized = scaler.fit_transform(signal_reshaped).flatten()
    return normalized

# Hàm lọc nhiễu
def notch_filter(data, notch_freq=50.0, fs=125, quality_factor=30): # fs=125 theo dữ liệu BIDMC
    nyq = 0.5 * fs
    w0 = notch_freq / nyq
    if w0 >= 1.0: # Tần số notch không được lớn hơn hoặc bằng Nyquist
         print(f"Warning: Notch frequency {notch_freq}Hz is too high for sampling rate {fs}Hz. Skipping notch filter.")
         return data
    b, a = iirnotch(w0, quality_factor)
    return filtfilt(b, a, data)

def butter_bandpass_filter(data, lowcut=0.5, highcut=8.0, fs=125, order=4): # Giảm order một chút
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    # Đảm bảo low < high và cả hai đều < 1.0
    low = max(0.01, low) # Tránh tần số quá thấp
    high = min(0.99, high) # Tránh tần số quá cao
    if low >= high:
        print(f"Warning: Invalid frequency range [{lowcut}, {highcut}] for sampling rate {fs}. Skipping bandpass filter.")
        return data
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, data)

# Hàm chia đoạn
def segment_signal(signal, segment_length, overlap=0.5): # overlap=0.5 là mặc định tốt
    step = int(segment_length * (1 - overlap))
    segments = []
    if step <= 0: # Tránh step = 0 nếu overlap >= 1
        step = 1
    for i in range(0, len(signal) - segment_length + 1, step):
        segments.append(signal[i:i + segment_length])
    # Kiểm tra xem có tạo được segment nào không
    if not segments:
        print(f"Warning: Could not create segments for signal of length {len(signal)} with segment_length {segment_length} and overlap {overlap}.")
    return np.array(segments)

# Hàm trích xuất đặc trưng HR/RR trung bình (Giữ nguyên logic cũ, xem xét cải tiến nếu có thể)
def extract_mean_hr_rr(record):
    hr_values_list = []
    rr_values_list = []
    try:
        params = record['ref'][0, 0]['params'][0, 0]
        hr_data = params['hr'][0]
        rr_data = params['rr'][0]

        # Xử lý HR
        if hasattr(hr_data, 'dtype') and hr_data.dtype.names is not None and 'v' in hr_data.dtype.names:
            hr_values_raw = hr_data['v']
        else:
            hr_values_raw = hr_data
        for item in hr_values_raw:
             # Xử lý mảng lồng nhau có thể rỗng hoặc chứa giá trị đơn lẻ
             val = item[0] if isinstance(item, (list, np.ndarray)) and len(item) > 0 else item
             if np.isscalar(val) and not np.isnan(val):
                 hr_values_list.append(float(val))

        # Xử lý RR
        if hasattr(rr_data, 'dtype') and rr_data.dtype.names is not None and 'v' in rr_data.dtype.names:
            rr_values_raw = rr_data['v']
        else:
            rr_values_raw = rr_data
        for item in rr_values_raw:
            val = item[0] if isinstance(item, (list, np.ndarray)) and len(item) > 0 else item
            if np.isscalar(val) and not np.isnan(val):
                 rr_values_list.append(float(val))

        if not hr_values_list or not rr_values_list:
            return None, None # Trả về None nếu không có đủ dữ liệu hợp lệ

        return np.mean(hr_values_list), np.mean(rr_values_list)

    except Exception as e:
        # print(f"Minor error extracting HR/RR: {e}") # Gỡ lỗi nếu cần
        return None, None

# Tham số tiền xử lý
fs = 125
segment_length = 8 * fs # 1000 samples
overlap = 0.5
lowcut = 0.5
highcut = 8.0
notch_freq = 50.0 # Tần số nhiễu điện lưới (có thể là 60Hz ở một số nơi)

# Danh sách lưu trữ
all_ppg_segments = []
all_hr_conditions = []
all_rr_conditions = []

valid_records_count = 0
# Vòng lặp xử lý từng bản ghi
for i in range(len(data)):
    try:
        record = data[i]
        # Lấy tín hiệu PPG
        ppg_signal_raw = record['ppg'][0, 0]['v'].flatten().astype(np.float64) # Đảm bảo float64 cho lọc

        # Lấy HR, RR trung bình cho bản ghi này
        hr_mean, rr_mean = extract_mean_hr_rr(record)
        if hr_mean is None or rr_mean is None:
            print(f"Skipping record {i}: Insufficient HR/RR data.")
            continue

        # 1. Lọc Notch
        ppg_notched = notch_filter(ppg_signal_raw, notch_freq=notch_freq, fs=fs)
        # 2. Lọc Bandpass
        ppg_filtered = butter_bandpass_filter(ppg_notched, lowcut=lowcut, highcut=highcut, fs=fs)
        # 3. Chuẩn hóa MinMax về [0, 1]
        ppg_normalized = normalize_signal_minmax(ppg_filtered)
        # 4. Chia đoạn
        segments = segment_signal(ppg_normalized, segment_length, overlap)

        if segments.size > 0:
            num_segments = segments.shape[0]
            all_ppg_segments.extend(segments)
            # Gán cùng HR/RR trung bình cho tất cả segment của bản ghi này
            all_hr_conditions.extend([hr_mean] * num_segments)
            all_rr_conditions.extend([rr_mean] * num_segments)
            valid_records_count += 1
        else:
             print(f"Skipping record {i}: No segments generated after processing.")

    except Exception as e:
        print(f"Error processing record {i}: {e}")

print(f"\nĐã xử lý thành công {valid_records_count}/{len(data)} bản ghi.")

if not all_ppg_segments:
    raise ValueError("Không có dữ liệu nào được xử lý thành công. Vui lòng kiểm tra dữ liệu đầu vào và quá trình tiền xử lý.")

# Chuyển thành Numpy arrays
X_data = np.array(all_ppg_segments).astype(np.float32)
hr_data = np.array(all_hr_conditions).astype(np.float32).reshape(-1, 1) # Reshape cho Scaler
rr_data = np.array(all_rr_conditions).astype(np.float32).reshape(-1, 1) # Reshape cho Scaler

print(f"Tổng số đoạn tín hiệu PPG: {X_data.shape[0]}")
print(f"Kích thước segment PPG: {X_data.shape[1]}")
print(f"Số lượng giá trị HR: {hr_data.shape[0]}")
print(f"Số lượng giá trị RR: {rr_data.shape[0]}")

# Chia train/test (90/10)
X_train, X_test, hr_train_raw, hr_test_raw, rr_train_raw, rr_test_raw = train_test_split(
    X_data, hr_data, rr_data, test_size=0.1, random_state=42
)

print(f"Kích thước tập huấn luyện: {X_train.shape[0]}")
print(f"Kích thước tập kiểm thử: {X_test.shape[0]}")

# ---> CHUẨN HÓA ĐIỀU KIỆN HR/RR <---
print("\nChuẩn hóa điều kiện HR và RR...")
hr_scaler = StandardScaler()
hr_train_scaled = hr_scaler.fit_transform(hr_train_raw)
hr_test_scaled = hr_scaler.transform(hr_test_raw)
joblib.dump(hr_scaler, os.path.join(processed_data_path, 'hr_scaler.gz')) # Lưu scaler HR
print("Đã lưu HR scaler.")

rr_scaler = StandardScaler()
rr_train_scaled = rr_scaler.fit_transform(rr_train_raw)
rr_test_scaled = rr_scaler.transform(rr_test_raw)
joblib.dump(rr_scaler, os.path.join(processed_data_path, 'rr_scaler.gz')) # Lưu scaler RR
print("Đã lưu RR scaler.")

# Tạo mảng điều kiện cuối cùng đã chuẩn hóa
condition_train = np.column_stack((hr_train_scaled.flatten(), rr_train_scaled.flatten())).astype(np.float32)
condition_test = np.column_stack((hr_test_scaled.flatten(), rr_test_scaled.flatten())).astype(np.float32)

print("Đã chuẩn hóa và kết hợp điều kiện HR/RR.")
print(f"Shape condition_train: {condition_train.shape}")
print(f"Shape condition_test: {condition_test.shape}")

# Lưu dữ liệu đã xử lý
np.save(os.path.join(processed_data_path, 'X_train.npy'), X_train)
np.save(os.path.join(processed_data_path, 'X_test.npy'), X_test)
np.save(os.path.join(processed_data_path, 'condition_train.npy'), condition_train)
np.save(os.path.join(processed_data_path, 'condition_test.npy'), condition_test)
# Lưu cả giá trị gốc để tham khảo nếu cần
np.save(os.path.join(processed_data_path, 'hr_train_raw.npy'), hr_train_raw)
np.save(os.path.join(processed_data_path, 'hr_test_raw.npy'), hr_test_raw)
np.save(os.path.join(processed_data_path, 'rr_train_raw.npy'), rr_train_raw)
np.save(os.path.join(processed_data_path, 'rr_test_raw.npy'), rr_test_raw)
print("Đã lưu các file dữ liệu đã xử lý vào:", processed_data_path)

# (Tùy chọn) Vẽ biểu đồ phân phối HR/RR gốc và các segment mẫu như trước
# ... (code vẽ biểu đồ có thể thêm vào đây) ...

print("\nTiền xử lý dữ liệu hoàn tất.")
# --- Hết Phần Preprocess ---


# --- Phần CVAE Model Definition ---
print("\nĐịnh nghĩa mô hình CVAE...")

# Tải dữ liệu (nếu cần chạy riêng section này)
# X_train = np.load(os.path.join(processed_data_path, 'X_train.npy'))
# X_test = np.load(os.path.join(processed_data_path, 'X_test.npy'))
# condition_train = np.load(os.path.join(processed_data_path, 'condition_train.npy'))
# condition_test = np.load(os.path.join(processed_data_path, 'condition_test.npy'))

# Tham số mô hình (Giữ nguyên hoặc tinh chỉnh)
input_dim = X_train.shape[1] # 1000
condition_dim = 2
latent_dim = 32
# hidden_units_dense_encoder = [256, 128, 64] # Units cho lớp Dense trong Encoder
# hidden_units_dense_decoder = [64, 128, 256] # Units cho lớp Dense trong Decoder
# Tham số huấn luyện
batch_size = 64
epochs = 500 # Giảm số epoch tối đa, dựa vào EarlyStopping
beta_max = 1.5 # Giảm beta max một chút
warmup_epochs = 50 # Tăng thời gian warmup
learning_rate = 0.0005 # Giữ LR này

# Lớp Sampling (Giữ nguyên)
class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# Encoder (Kiến trúc Conv1D)
def build_encoder(input_dim, condition_dim, latent_dim, hidden_units_dense=[256, 128]): # Giảm bớt lớp Dense
    encoder_inputs = layers.Input(shape=(input_dim,), name='encoder_input')
    x = layers.Reshape((input_dim, 1))(encoder_inputs)

    x = layers.Conv1D(filters=64, kernel_size=5, strides=1, activation='swish', padding='same')(x)
    x = layers.BatchNormalization()(x) # Thêm BN
    x = layers.MaxPooling1D(pool_size=2)(x) # 1000 -> 500

    x = layers.Conv1D(filters=128, kernel_size=5, strides=1, activation='swish', padding='same')(x)
    x = layers.BatchNormalization()(x) # Thêm BN
    x = layers.MaxPooling1D(pool_size=2)(x) # 500 -> 250

    x = layers.Conv1D(filters=256, kernel_size=5, strides=1, activation='swish', padding='same')(x)
    x = layers.BatchNormalization()(x) # Thêm BN
    x = layers.MaxPooling1D(pool_size=2)(x) # 250 -> 125

    shape_before_flattening = tf.keras.backend.int_shape(x)[1:] # (125, 256)
    x = layers.Flatten()(x)

    condition_inputs = layers.Input(shape=(condition_dim,), name='condition_input')
    # Nhúng điều kiện vào không gian chiều cao hơn
    cond_dense = layers.Dense(16, activation='relu')(condition_inputs) # Nhúng điều kiện

    x = layers.Concatenate()([x, cond_dense]) # Ghép đặc trưng PPG và điều kiện đã nhúng

    for units in hidden_units_dense:
        x = layers.Dense(units, activation='swish')(x)
        x = layers.Dropout(0.3)(x) # Thêm Dropout

    z_mean = layers.Dense(latent_dim, name='z_mean')(x)
    z_log_var = layers.Dense(latent_dim, name='z_log_var')(x)
    z = Sampling()([z_mean, z_log_var])

    encoder = Model([encoder_inputs, condition_inputs], [z_mean, z_log_var, z], name='encoder')
    # Trả về cả shape để dùng cho decoder
    return encoder, shape_before_flattening

# Decoder (Kiến trúc Conv1DTranspose)
def build_decoder(latent_dim, condition_dim, input_dim, shape_before_flattening, hidden_units_dense=[128, 256]): # Giảm bớt lớp Dense
    latent_inputs = layers.Input(shape=(latent_dim,), name='latent_input')
    condition_inputs = layers.Input(shape=(condition_dim,), name='condition_input')

    # Nhúng điều kiện tương tự encoder
    cond_dense = layers.Dense(16, activation='relu')(condition_inputs)

    x = layers.Concatenate()([latent_inputs, cond_dense])

    for units in reversed(hidden_units_dense):
         x = layers.Dense(units, activation='swish')(x)
         # Có thể thêm Dropout ở đây nếu cần

    target_shape_units = np.prod(shape_before_flattening)
    x = layers.Dense(target_shape_units, activation='swish')(x)
    x = layers.Reshape(shape_before_flattening)(x) # Shape: (None, 125, 256)

    x = layers.Conv1DTranspose(128, kernel_size=5, strides=2, padding='same', activation='swish')(x) # 125 -> 250
    x = layers.BatchNormalization()(x) # Thêm BN

    x = layers.Conv1DTranspose(64, kernel_size=5, strides=2, padding='same', activation='swish')(x) # 250 -> 500
    x = layers.BatchNormalization()(x) # Thêm BN

    # Lớp cuối cùng: filters=1, activation='sigmoid' (SỬA LỖI)
    x = layers.Conv1DTranspose(1, kernel_size=5, strides=2, padding='same', activation='sigmoid')(x) # 500 -> 1000

    decoder_outputs = layers.Reshape((input_dim,))(x)
    decoder = Model([latent_inputs, condition_inputs], decoder_outputs, name='decoder')
    return decoder

# Lớp CVAE với KL Annealing (Cập nhật metric tính toán)
class CVAE_Combined(Model):
    def __init__(self, encoder, decoder, latent_dim, beta_max, warmup_epochs, **kwargs):
        super(CVAE_Combined, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.latent_dim = latent_dim
        self.beta_max = tf.constant(beta_max, dtype=tf.float32)
        self.warmup_epochs = tf.constant(warmup_epochs, dtype=tf.float32)
        self.epoch_counter = tf.Variable(0.0, trainable=False, name="epoch_counter", dtype=tf.float32)

        # Trackers
        self.total_loss_tracker = tf.keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = tf.keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = tf.keras.metrics.Mean(name="kl_loss")
        self.beta_tracker = tf.keras.metrics.Mean(name="beta") # Theo dõi beta thực tế

    @property
    def metrics(self):
        # Đảm bảo trả về đúng các tracker đã định nghĩa
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
            self.beta_tracker
        ]

    # Hàm tính beta dựa trên epoch counter
    def get_current_beta(self):
         # Chia tránh zero division
         safe_warmup_epochs = tf.maximum(self.warmup_epochs, 1.0)
         # epoch_counter tăng từ 0.0
         current_progress = (self.epoch_counter + 1.0) / safe_warmup_epochs
         beta = tf.minimum(self.beta_max, self.beta_max * current_progress)
         return beta

    def train_step(self, data):
        x, condition = data
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder([x, condition], training=True)
            reconstruction = self.decoder([z, condition], training=True)

            # Reconstruction loss (MSE)
            reconstruction_loss = tf.reduce_mean(
                 tf.reduce_sum(
                     tf.keras.losses.mse(x, reconstruction), axis=1 # Sum trên chiều thời gian
                 )
            )
            # KL loss
            kl_loss = -0.5 * tf.reduce_mean(
                tf.reduce_sum(
                    1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=1 # Sum trên chiều latent
                )
            )

            # Lấy beta hiện tại
            current_beta = self.get_current_beta()
            total_loss = reconstruction_loss + current_beta * kl_loss

        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

        # Cập nhật trackers
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        self.beta_tracker.update_state(current_beta) # Cập nhật beta tracker

        # Trả về dict các metric hiện tại
        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        x, condition = data
        z_mean, z_log_var, z = self.encoder([x, condition], training=False)
        reconstruction = self.decoder([z, condition], training=False)

        reconstruction_loss = tf.reduce_mean(
             tf.reduce_sum(tf.keras.losses.mse(x, reconstruction), axis=1)
        )
        kl_loss = -0.5 * tf.reduce_mean(
            tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=1)
        )

        current_beta = self.get_current_beta() # Dùng beta tương ứng epoch hiện tại
        total_loss = reconstruction_loss + current_beta * kl_loss

        # Cập nhật trackers (quan trọng cho validation metrics)
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        self.beta_tracker.update_state(current_beta)

        return {m.name: m.result() for m in self.metrics}

    # Không cần hàm call riêng nếu train_step/test_step đã đủ
    # def call(self, inputs, training=False): # Thêm training flag nếu cần BN/Dropout
    #     x, condition = inputs
    #     z_mean, z_log_var, z = self.encoder([x, condition], training=training)
    #     reconstruction = self.decoder([z, condition], training=training)
    #     return reconstruction

    def generate(self, condition, z=None, noise_scale=1.0): # noise_scale=1.0 là chuẩn
         if z is None:
             z = tf.random.normal(shape=(tf.shape(condition)[0], self.latent_dim)) * noise_scale
         # Cần đảm bảo condition có đúng shape (batch_size, condition_dim)
         if tf.rank(condition) == 1:
             condition = tf.expand_dims(condition, axis=0)
         return self.decoder([z, condition], training=False) # Luôn dùng training=False khi generate

# Callback để cập nhật epoch counter (QUAN TRỌNG cho beta annealing)
class EpochCounterCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        # Cập nhật biến epoch_counter của mô hình
        if hasattr(self.model, 'epoch_counter'):
            self.model.epoch_counter.assign(tf.cast(epoch, tf.float32))
            # tf.print("Epoch Counter Updated:", self.model.epoch_counter) # Debug nếu cần

# Xây dựng mô hình
print("\nXây dựng Encoder và Decoder...")
encoder, shape_enc = build_encoder(input_dim, condition_dim, latent_dim)
decoder = build_decoder(latent_dim, condition_dim, input_dim, shape_enc)
encoder.summary(line_length=100)
decoder.summary(line_length=100)

print("\nXây dựng và biên dịch mô hình CVAE kết hợp...")
cvae_combined = CVAE_Combined(encoder, decoder, latent_dim, beta_max, warmup_epochs)
cvae_combined.compile(optimizer=Adam(learning_rate=learning_rate))
print("Biên dịch hoàn tất.")

# --- Hết Phần Model Definition ---

# --- Phần Training ---
print("\nBắt đầu Huấn luyện mô hình CVAE kết hợp...")

# Thiết lập Callbacks (SỬ DỤNG DANH SÁCH ĐẦY ĐỦ)
log_dir = os.path.join(model_path, "logs_combined", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

checkpoint_path = os.path.join(model_path, "cvae_combined_best.weights.h5") # Đổi tên file checkpoint
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    monitor='val_total_loss', # Theo dõi val_total_loss
    mode='min',
    save_best_only=True, # Chỉ lưu model tốt nhất
    verbose=1
)

early_stopping_callback = EarlyStopping(
    monitor='val_total_loss',
    patience=50,  # Tăng patience lên một chút nữa
    restore_best_weights=True, # QUAN TRỌNG: Khôi phục trọng số tốt nhất khi dừng
    verbose=1
)

reduce_lr_callback = ReduceLROnPlateau(
    monitor='val_total_loss',
    factor=0.2, # Giảm LR mạnh hơn
    patience=20, # Kiên nhẫn hơn trước khi giảm LR
    min_lr=1e-7, # LR tối thiểu
    verbose=1
)

# DANH SÁCH CALLBACKS ĐẦY ĐỦ SẼ ĐƯỢC SỬ DỤNG
combined_callbacks = [
    EpochCounterCallback(), # Cập nhật epoch cho beta annealing
    tensorboard_callback,
    checkpoint_callback,    # Lưu model tốt nhất
    early_stopping_callback,# Dừng sớm và khôi phục trọng số tốt nhất
    reduce_lr_callback     # Giảm learning rate
]

# Tạo dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, condition_train)).shuffle(buffer_size=X_train.shape[0]).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, condition_test)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Huấn luyện
start_time = time.time()
history = cvae_combined.fit(
    train_dataset,
    epochs=epochs, # Số epochs lớn, EarlyStopping sẽ kiểm soát
    validation_data=test_dataset,
    callbacks=combined_callbacks # <-- SỬ DỤNG CALLBACKS ĐẦY ĐỦ
)
training_time = time.time() - start_time

print(f"\nHuấn luyện hoàn tất trong {training_time:.2f} giây.")
print(f"Trọng số tốt nhất đã được lưu tại (hoặc khôi phục vào model): {checkpoint_path}")

# Lưu trọng số cuối cùng (tùy chọn, vì EarlyStopping đã khôi phục best weights)
# cvae_combined.save_weights(os.path.join(model_path, 'cvae_combined_final.weights.h5'))

# Vẽ biểu đồ huấn luyện
print('\nVẽ biểu đồ quá trình huấn luyện...')
plt.figure(figsize=(20, 5)) # Rộng hơn để chứa 5 plot

# Loss tổng thể
plt.subplot(1, 5, 1)
plt.plot(history.history['total_loss'], label='Train Total Loss')
plt.plot(history.history['val_total_loss'], label='Val Total Loss')
plt.title('Total Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

# Loss tái tạo
plt.subplot(1, 5, 2)
plt.plot(history.history['reconstruction_loss'], label='Train Recon Loss')
plt.plot(history.history['val_reconstruction_loss'], label='Val Recon Loss')
plt.title('Reconstruction Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

# Loss KL
plt.subplot(1, 5, 3)
plt.plot(history.history['kl_loss'], label='Train KL Loss')
plt.plot(history.history['val_kl_loss'], label='Val KL Loss')
plt.title('KL Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

# Beta
plt.subplot(1, 5, 4)
if 'beta' in history.history: # Kiểm tra xem beta có trong history không (phụ thuộc Keras version và cách metric được log)
     # Lấy beta từ val_beta nếu có, hoặc train_beta
     beta_key = 'val_beta' if 'val_beta' in history.history else 'beta'
     plt.plot(history.history[beta_key], label='Beta Value', color='orange')
     plt.title('Beta Annealing')
     plt.xlabel('Epoch')
     plt.ylabel('Beta')
     plt.legend()
     plt.grid(True, alpha=0.3)
else:
     plt.title('Beta (Not Logged)')


# Learning Rate
plt.subplot(1, 5, 5)
if 'lr' in history.history:
    plt.plot(history.history['lr'], label='Learning Rate', color='purple')
    plt.title('Learning Rate Schedule')
    plt.xlabel('Epoch')
    plt.ylabel('Learning Rate')
    plt.legend()
    plt.grid(True, alpha=0.3)
else:
     plt.title('LR (Not Logged)')


plt.tight_layout()
plt.savefig(os.path.join(figures_path, 'training_history_combined.png'))
plt.show()
plt.close()

print("Đã lưu biểu đồ huấn luyện.")
# --- Hết Phần Training ---


# --- Phần Test & Evaluation ---
print("\nBắt đầu Kiểm tra và Đánh giá mô hình...")

# >>> QUAN TRỌNG: Load trọng số tốt nhất đã lưu <<<
# Ngay cả khi EarlyStopping có restore_best_weights=True, việc load lại đảm bảo chắc chắn
if os.path.exists(checkpoint_path):
    print(f"Đang tải trọng số tốt nhất từ: {checkpoint_path}")
    try:
        # Cần build model trước khi load weights nếu chưa được build đầy đủ
        # Thử build bằng cách gọi với dữ liệu mẫu
        _ = cvae_combined([X_test[:1], condition_test[:1]])
        cvae_combined.load_weights(checkpoint_path)
        print("Đã tải trọng số thành công.")
    except Exception as e:
        print(f"Lỗi khi tải trọng số: {e}. Sử dụng trọng số hiện tại trong model.")
else:
    print(f"Không tìm thấy file checkpoint: {checkpoint_path}. Sử dụng trọng số cuối cùng trong model.")


# 1. Tái tạo tín hiệu từ tập Test
print("Tái tạo tín hiệu từ tập test...")
num_samples_to_show = 10
X_test_subset = X_test[:num_samples_to_show]
condition_test_subset = condition_test[:num_samples_to_show]

# Lấy z_mean, z_log_var, z từ encoder
z_mean, z_log_var, z = cvae_combined.encoder.predict([X_test_subset, condition_test_subset])
# Tái tạo từ z (sử dụng z thay vì z_mean để xem xét cả phần ngẫu nhiên)
reconstructed_ppg = cvae_combined.decoder.predict([z, condition_test_subset])
# Hoặc tái tạo trực tiếp bằng cách gọi model (nếu có hàm call)
# reconstructed_ppg = cvae_combined.predict([X_test_subset, condition_test_subset])

# Load lại HR/RR gốc để hiển thị tiêu đề cho dễ hiểu
hr_test_raw_subset = np.load(os.path.join(processed_data_path, 'hr_test_raw.npy'))[:num_samples_to_show]
rr_test_raw_subset = np.load(os.path.join(processed_data_path, 'rr_test_raw.npy'))[:num_samples_to_show]

plt.figure(figsize=(15, 4 * num_samples_to_show // 2))
for i in range(num_samples_to_show):
    plt.subplot(num_samples_to_show // 2, 2, i + 1)
    plt.plot(X_test_subset[i], label='Original', alpha=0.8)
    plt.plot(reconstructed_ppg[i], label='Reconstructed', alpha=0.8, linestyle='--')
    plt.title(f'Test Sample {i+1} (HR={hr_test_raw_subset[i,0]:.1f}, RR={rr_test_raw_subset[i,0]:.1f})')
    plt.xlabel('Sample')
    plt.ylabel('Amplitude')
    plt.legend()
    plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(figures_path, 'reconstruction_comparison.png'))
plt.show()
plt.close()

# 2. Phân tích tần số so sánh
print("Phân tích phổ tần số...")
def analyze_frequency_spectrum(signal, fs):
    n = len(signal)
    if n == 0: return np.array([]), np.array([])
    yf = fft(signal)
    xf = fftfreq(n, 1/fs)[:n//2]
    yf_abs = 2.0/n * np.abs(yf[0:n//2])
    return xf, yf_abs

num_fft_samples = 5
plt.figure(figsize=(15, 5 * num_fft_samples))
for i in range(num_fft_samples):
    # Original
    xf_orig, yf_orig = analyze_frequency_spectrum(X_test_subset[i], fs)
    # Reconstructed
    xf_recon, yf_recon = analyze_frequency_spectrum(reconstructed_ppg[i], fs)

    plt.subplot(num_fft_samples, 2, 2 * i + 1)
    if xf_orig.size > 0:
      plt.plot(xf_orig, yf_orig)
      peaks_orig_idx = np.argsort(yf_orig)[-3:] # Top 3 peaks
      plt.plot(xf_orig[peaks_orig_idx], yf_orig[peaks_orig_idx], 'ro', label='Peaks')
      for idx in peaks_orig_idx:
           plt.text(xf_orig[idx], yf_orig[idx], f'{xf_orig[idx]:.2f}Hz', fontsize=9)
    plt.title(f'Original FFT (Sample {i+1})')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Amplitude')
    plt.xlim(0, 10) # Giới hạn tần số
    plt.grid(True, alpha=0.3)
    plt.legend()


    plt.subplot(num_fft_samples, 2, 2 * i + 2)
    if xf_recon.size > 0:
      plt.plot(xf_recon, yf_recon, color='orange')
      peaks_recon_idx = np.argsort(yf_recon)[-3:] # Top 3 peaks
      plt.plot(xf_recon[peaks_recon_idx], yf_recon[peaks_recon_idx], 'ro', label='Peaks')
      for idx in peaks_recon_idx:
           plt.text(xf_recon[idx], yf_recon[idx], f'{xf_recon[idx]:.2f}Hz', fontsize=9)
    plt.title(f'Reconstructed FFT (Sample {i+1})')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Amplitude')
    plt.xlim(0, 10)
    plt.grid(True, alpha=0.3)
    plt.legend()

plt.tight_layout()
plt.savefig(os.path.join(figures_path, 'fft_comparison.png'))
plt.show()
plt.close()

# 3. Tính toán Metrics định lượng
print("Tính toán các chỉ số đánh giá...")
mse_list, psnr_list, corr_list = [], [], []

def calculate_psnr(original, generated, max_pixel=1.0): # Max pixel là 1.0 do MinMax về [0,1]
    mse = mean_squared_error(original, generated)
    if mse == 0:
        return float('inf')
    return 20 * np.log10(max_pixel / np.sqrt(mse))

def calculate_corr(original, generated):
     # Xử lý trường hợp tín hiệu là hằng số (std=0)
     if np.std(original) == 0 or np.std(generated) == 0:
         return 0.0 if np.allclose(original, generated) else np.nan
     return np.corrcoef(original, generated)[0, 1]


# Tính toán trên toàn bộ tập test hoặc một phần lớn hơn
num_eval_samples = min(1000, X_test.shape[0]) # Đánh giá trên 1000 mẫu hoặc toàn bộ tập test
X_eval = X_test[:num_eval_samples]
cond_eval = condition_test[:num_eval_samples]

# Tái tạo cho tập đánh giá
print(f"Tái tạo {num_eval_samples} mẫu để đánh giá...")
eval_z_mean, _, eval_z = cvae_combined.encoder.predict([X_eval, cond_eval])
eval_reconstructed = cvae_combined.decoder.predict([eval_z, cond_eval]) # Dùng z để có cả phần ngẫu nhiên

for i in range(num_eval_samples):
    orig = X_eval[i]
    recon = eval_reconstructed[i]
    mse_list.append(mean_squared_error(orig, recon))
    psnr_list.append(calculate_psnr(orig, recon))
    corr_list.append(calculate_corr(orig, recon))

# Loại bỏ NaN trong corr nếu có
corr_list_valid = [c for c in corr_list if not np.isnan(c)]

results_summary = {
    "MSE": {"Mean": np.mean(mse_list), "Std": np.std(mse_list), "Min": np.min(mse_list), "Max": np.max(mse_list)},
    "PSNR": {"Mean": np.mean(psnr_list), "Std": np.std(psnr_list), "Min": np.min(psnr_list), "Max": np.max(psnr_list)},
    "Correlation": {"Mean": np.mean(corr_list_valid), "Std": np.std(corr_list_valid), "Min": np.min(corr_list_valid), "Max": np.max(corr_list_valid)}
}

print("\n--- Kết quả Đánh giá Định lượng ---")
for metric, stats in results_summary.items():
     print(f"{metric}: Mean={stats['Mean']:.4f} +/- {stats['Std']:.4f} (Min={stats['Min']:.4f}, Max={stats['Max']:.4f})")

# Lưu kết quả vào file
results_file_path = os.path.join(results_path, "evaluation_summary.txt")
with open(results_file_path, "w") as f:
    f.write("KẾT QUẢ ĐÁNH GIÁ MÔ HÌNH CVAE KẾT HỢP\n")
    f.write("=======================================\n\n")
    f.write(f"Đánh giá trên {num_eval_samples} mẫu.\n\n")
    for metric, stats in results_summary.items():
        f.write(f"{metric}:\n")
        f.write(f"  - Mean: {stats['Mean']:.4f}\n")
        f.write(f"  - Std Dev: {stats['Std']:.4f}\n")
        f.write(f"  - Min: {stats['Min']:.4f}\n")
        f.write(f"  - Max: {stats['Max']:.4f}\n\n")
print(f"Đã lưu kết quả đánh giá vào: {results_file_path}")

# --- Hết Phần Test & Evaluation ---


# --- Phần Prediction (Sinh tín hiệu mới) ---
print("\nBắt đầu Sinh tín hiệu PPG mới...")

# Load lại scaler đã lưu trong quá trình tiền xử lý
try:
    hr_scaler_loaded = joblib.load(os.path.join(processed_data_path, 'hr_scaler.gz'))
    rr_scaler_loaded = joblib.load(os.path.join(processed_data_path, 'rr_scaler.gz'))
    print("Đã tải thành công HR và RR scalers.")
except FileNotFoundError:
    print("Lỗi: Không tìm thấy file scaler đã lưu. Không thể thực hiện dự đoán với giá trị gốc.")
    # Thoát hoặc xử lý lỗi khác
    hr_scaler_loaded = None
    rr_scaler_loaded = None

if hr_scaler_loaded and rr_scaler_loaded:
    # Tạo lưới các giá trị HR, RR gốc mong muốn
    hr_raw_values = np.linspace(60, 120, 5) # Ví dụ: 60, 75, 90, 105, 120 bpm
    rr_raw_values = np.linspace(8, 20, 5) # Ví dụ: 8, 11, 14, 17, 20 breaths/min

    # Chuẩn hóa các giá trị này bằng scaler đã load
    hr_scaled_pred = hr_scaler_loaded.transform(hr_raw_values.reshape(-1, 1))
    rr_scaled_pred = rr_scaler_loaded.transform(rr_raw_values.reshape(-1, 1))

    # Tạo các cặp điều kiện đã chuẩn hóa
    conditions_pred_list = []
    conditions_raw_list = [] # Lưu lại giá trị gốc để hiển thị
    for hr_s, hr_r in zip(hr_scaled_pred, hr_raw_values):
        for rr_s, rr_r in zip(rr_scaled_pred, rr_raw_values):
            conditions_pred_list.append([hr_s[0], rr_s[0]])
            conditions_raw_list.append([hr_r, rr_r])

    conditions_pred_tensor = tf.constant(conditions_pred_list, dtype=tf.float32)
    conditions_raw_array = np.array(conditions_raw_list)

    print(f"\nSinh {conditions_pred_tensor.shape[0]} tín hiệu PPG với các điều kiện HR/RR cụ thể (đã chuẩn hóa)...")
    # Sinh tín hiệu (có thể thêm nhiễu z để đa dạng hóa)
    generated_ppg_specific = cvae_combined.generate(conditions_pred_tensor, noise_scale=0.7).numpy() # Thêm chút nhiễu

    # Vẽ một số tín hiệu đã tạo
    num_pred_to_show = min(10, generated_ppg_specific.shape[0])
    indices_to_show = np.random.choice(generated_ppg_specific.shape[0], num_pred_to_show, replace=False)

    plt.figure(figsize=(15, 4 * num_pred_to_show // 2))
    plot_count = 1
    for idx in indices_to_show:
         if plot_count > num_pred_to_show: break
         plt.subplot(num_pred_to_show // 2, 2, plot_count)
         plt.plot(generated_ppg_specific[idx])
         hr_orig = conditions_raw_array[idx, 0]
         rr_orig = conditions_raw_array[idx, 1]
         plt.title(f'Generated PPG (Target HR={hr_orig:.1f}, RR={rr_orig:.1f})')
         plt.xlabel('Sample')
         plt.ylabel('Amplitude')
         plt.grid(True, alpha=0.3)
         plot_count += 1
    plt.tight_layout()
    plt.savefig(os.path.join(figures_path, 'generated_signals_specific_conditions.png'))
    plt.show()
    plt.close()
    print("Đã lưu biểu đồ các tín hiệu sinh ra.")

# --- Hết Phần Prediction ---

print("\n--- QUÁ TRÌNH HOÀN TẤT ---")

Thiết lập đường dẫn và thư mục hoàn tất.
Đang tải dữ liệu từ file .mat...
Số lượng bản ghi: 53

Bắt đầu Tiền xử lý dữ liệu...
Skipping record 0: Insufficient HR/RR data.
Skipping record 1: Insufficient HR/RR data.
Skipping record 2: Insufficient HR/RR data.
Skipping record 3: Insufficient HR/RR data.
Skipping record 4: Insufficient HR/RR data.
Skipping record 5: Insufficient HR/RR data.
Skipping record 6: Insufficient HR/RR data.
Skipping record 7: Insufficient HR/RR data.
Skipping record 8: Insufficient HR/RR data.
Skipping record 9: Insufficient HR/RR data.
Skipping record 10: Insufficient HR/RR data.
Skipping record 11: Insufficient HR/RR data.
Skipping record 12: Insufficient HR/RR data.
Skipping record 13: Insufficient HR/RR data.
Skipping record 14: Insufficient HR/RR data.
Skipping record 15: Insufficient HR/RR data.
Skipping record 16: Insufficient HR/RR data.
Skipping record 17: Insufficient HR/RR data.
Skipping record 18: Insufficient HR/RR data.
Skipping record 19: Insuffici

ValueError: Không có dữ liệu nào được xử lý thành công. Vui lòng kiểm tra dữ liệu đầu vào và quá trình tiền xử lý.